$$
P(\mathbf{y}| \mathbf{X})
$$

In [15]:
import numpy as np
import pandas as pd
from scipy.stats import mode
from pandas import DataFrame
from numpy.typing import ArrayLike

In [2]:
X = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1],
])

In [3]:
y = np.array([
    0,
    0,
    1,
    0,
    1,
    0,
    0,
    0
])

$$
x_1' \cdot x_2 \cdot x_3' + x_1 \cdot x_2' \cdot x_3'
$$

In [5]:
def predict(x: ArrayLike) -> ArrayLike:
    x1, x2, x3 = x[:, 0], x[:, 1], x[:, 2]
    return (
        (~ x1 & x2 & ~x3 ) |
        (x1 & ~x2 & ~x3)
    )

In [6]:
y_pred = predict(X)

In [7]:
y_pred

array([0, 0, 1, 0, 1, 0, 0, 0])

In [8]:
y

array([0, 0, 1, 0, 1, 0, 0, 0])

In [89]:
X = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 1, 1, 0],
    [1, 0, 0, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [1, 1, 1, 1],
    [0, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 1],
    [1, 1, 0, 0],
    [1, 1, 1, 0],
    [1, 1, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 1],
    [0, 1, 0, 0],
    [0, 1, 0, 1],
], dtype=np.bool8)
df = pd.DataFrame(data=X, columns=["x1", "x2", "x3", "y"])

In [90]:
df

,x1,x2,x3,y
0,False,False,False,False
1,False,False,True,False
2,False,True,False,True
3,False,True,True,False
4,True,False,False,False
5,True,False,True,True
6,True,True,False,False
7,True,True,True,True
8,False,False,False,False
9,False,False,False,True


In [91]:
mode([0, 0, 0, 1, 1]).mode

<ipython-input-91-247409047fc4>:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode([0, 0, 0, 1, 1]).mode


array([0])

In [99]:
(
    df
    .groupby(["x1", "x2", "x3"])
    .agg({"y": lambda i: float(mode(i).mode)})
    .reset_index()
    .astype({"y": "int"})
)

<ipython-input-99-0d4934911eeb>:4: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  .agg({"y": lambda i: float(mode(i).mode)})
<ipython-input-99-0d4934911eeb>:4: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  .agg({"y": lambda i: float(mode(i).mode)})


,x1,x2,x3,y
0,False,False,False,1
1,False,False,True,0
2,False,True,False,1
3,False,True,True,0
4,True,False,False,0
5,True,False,True,1
6,True,True,False,0
7,True,True,True,0


In [100]:
def train(df: DataFrame) -> str:
    df_true = (
        df
        .groupby(["x1", "x2", "x3"])
        .agg({"y": lambda i: float(mode(i).mode)})
        .reset_index()
        .astype({"y": "int"})
        .query("y == 1")
        .drop(columns=["y"])
    )
    all_rules = []
    for i, row in df_true.iterrows():
        rules = []
        for index in row.index:
            if row[index]:
                rules.append(index)
            else:
                rules.append(f"~{index}")
        all_rules.append(f"({' & '.join(rules)})")
    return " | ".join(all_rules)

In [101]:
rule = train(df)

<ipython-input-100-6e757d34d0b7>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  .agg({"y": lambda i: float(mode(i).mode)})
<ipython-input-100-6e757d34d0b7>:5: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  .agg({"y": lambda i: float(mode(i).mode)})


In [102]:
rule

'(~x1 & ~x2 & ~x3) | (~x1 & x2 & ~x3) | (x1 & ~x2 & x3)'

In [108]:
def predict(X: ArrayLike, rule: str) -> ArrayLike:
    x1, x2, x3 = X[:, 0], X[:, 1], X[:, 2]
    y_pred = eval(rule)
    return y_pred.astype("int")

In [109]:
y_pred = predict(X, rule)

In [110]:
y_pred

array([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1])

In [111]:
y = df.y.values

In [112]:
(y == y_pred).mean()

0.7894736842105263